In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
path = r'C:\Users\Zarqa Ayub\Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
df = ords_prods_merge[:1000000]

### Step 2 find the aggregated mean of the "order_number" column grouped by "department_id" for the entire dataframe.

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number           
                      mean  min   max
department_id                        
1.0              15.457687  1.0  99.0
2.0              17.277920  1.0  99.0
3.0              17.179756  1.0  99.0
4.0              17.811403  1.0  99.0
5.0              15.213779  1.0  99.0
6.0              16.439806  1.0  99.0
7.0              17.225773  1.0  99.0
8.0              15.340520  1.0  99.0
9.0              15.895474  1.0  99.0
10.0             20.197148  1.0  99.0
11.0             16.170828  1.0  99.0
12.0             15.887622  1.0  99.0
13.0             16.583304  1.0  99.0
14.0             16.757377  1.0  99.0
15.0             16.165037  1.0  99.0
16.0             17.663250  1.0  99.0
17.0             15.694469  1.0  99.0
18.0             19.310397  1.0  99.0
19.0             17.177343  1.0  99.0
20.0             16.473447  1.0  99.0
21.0             22.902379  1.0  99.0

### Step 3: compare the result of the entire dataframe with that of the subset

In [6]:
# Agg the subset
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number           
                      mean  min   max
department_id                        
1.0              15.562844  1.0  99.0
2.0              17.518895  1.0  96.0
3.0              16.907152  1.0  99.0
4.0              17.537934  1.0  99.0
5.0              14.894521  1.0  99.0
6.0              16.658449  1.0  99.0
7.0              17.072954  1.0  99.0
8.0              14.942021  1.0  98.0
9.0              15.447580  1.0  99.0
10.0             18.681852  1.0  99.0
11.0             15.352278  1.0  99.0
12.0             14.327957  1.0  99.0
13.0             16.578690  1.0  99.0
14.0             16.666369  1.0  99.0
15.0             16.153766  1.0  99.0
16.0             17.768070  1.0  99.0
17.0             15.602964  1.0  99.0
18.0             19.674252  1.0  99.0
19.0             16.924344  1.0  99.0
20.0             16.292631  1.0  99.0
21.0             25.535596  1.0  98.0

### Step 4: Create a loyality flag for exsisting customers

In [7]:
# Create a new 'max_orders' column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

### Create a flag that assign a "loyality" label to user ID on its corresponding max order value

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
# Check frequencies of loyalty flags for accuracy
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    16382119
Loyal customer      10679887
New customer         5580444
NaN                       11
Name: count, dtype: int64

In [10]:
# check output
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,Busiest days,Busiest_period_of_day,max_order,loyalty_flag
0,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,28.0,6.0,11.0,3.0,5.0,0.0,both,Regularly busy,Most orders,33.0,Regular customer
1,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,30.0,6.0,17.0,20.0,1.0,1.0,both,Regularly busy,Average orders,33.0,Regular customer
2,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,2.0,0.0,21.0,6.0,20.0,0.0,both,Busiest days,Average orders,6.0,New customer
3,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,1.0,3.0,13.0,8.0,10.0,0.0,both,Least busy,Most orders,4.0,New customer
4,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,3.0,4.0,17.0,9.0,11.0,1.0,both,Least busy,Average orders,4.0,New customer


### Step 5: Determine whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers

In [11]:
ords_prods_merge[['user_id', 'loyalty_flag', 'max_order', 'prices']].head(30)

,user_id,loyalty_flag,max_order,prices
0,138.0,Regular customer,33.0,5.8
1,138.0,Regular customer,33.0,5.8
2,709.0,New customer,6.0,5.8
3,764.0,New customer,4.0,5.8
4,764.0,New customer,4.0,5.8
5,777.0,Regular customer,27.0,5.8
6,825.0,New customer,10.0,5.8
7,910.0,Regular customer,13.0,5.8
8,1052.0,Regular customer,21.0,5.8
9,1052.0,Regular customer,21.0,5.8


In [12]:
ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

0            8.0
1            8.0
2            8.0
3            9.0
4            9.0
            ... 
32642456     8.0
32642457     8.0
32642458     8.0
32642459     5.0
32642460    12.0
Name: days_since_prior_order, Length: 32642461, dtype: float16

Yes, on average there is a price difference between the new and loyal customer. On average, the new customer spends more than the loyal customer, though the loyal customer purchases more items.

### Step 6: Create a spending flag for each user based on the avergae price across all their orders


In [13]:
# group by the prices 
ords_prods_merge.groupby('prices')

In [14]:
# Split the data into groups by "loyalty_flag" and then calculate the mean for each "prices" column
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.379575
New customer,13.208206
Regular customer,12.616590


In [16]:
# Group data by "user_id" column, generate mean product price for each user, create "mean_price" for aggregation results
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
# Create flags to assign "spending" labels to user ID based on mean product price
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
# Check frequencies of spending flags for accuracy
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32001528
High spender      640922
NaN                   11
Name: count, dtype: int64

### Step 7: Create an order frequency flag that marks the regularity of a user's ordering behaviour according to the median in the "days_since_prior_order" column.

In [19]:
# Agg the dataframe
ords_prods_merge.groupby('user_id')['days_since_prior_order'].median()

user_id
1.0         20.0
2.0         13.0
3.0          9.0
4.0         19.0
5.0         11.0
            ... 
206205.0     8.0
206206.0     3.0
206207.0     8.0
206208.0     7.0
206209.0    18.0
Name: days_since_prior_order, Length: 206209, dtype: float16

In [20]:
# create a median column to put your reults in
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

In [21]:
# check output
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,...,add_to_cart_order,reordered,_merge,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days_since_prior_order
0,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,28.0,6.0,11.0,...,5.0,0.0,both,Regularly busy,Most orders,33.0,Regular customer,6.935811,Low spender,8.0
1,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,30.0,6.0,17.0,...,1.0,1.0,both,Regularly busy,Average orders,33.0,Regular customer,6.935811,Low spender,8.0
2,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,2.0,0.0,21.0,...,20.0,0.0,both,Busiest days,Average orders,6.0,New customer,7.930208,Low spender,8.0
3,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,1.0,3.0,13.0,...,10.0,0.0,both,Least busy,Most orders,4.0,New customer,4.972414,Low spender,9.0
4,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,3.0,4.0,17.0,...,11.0,1.0,both,Least busy,Average orders,4.0,New customer,4.972414,Low spender,9.0


In [22]:
#Assign flag conditions
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'Order frequency'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] <= 20) & (ords_prods_merge['median_days_since_prior_order'] > 10), 'Order frequency'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'Order frequency'] = 'Frequent customer'

In [23]:
# check output
ords_prods_merge.head(15)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,...,reordered,_merge,Busiest days,Busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days_since_prior_order,Order frequency
0,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,28.0,6.0,11.0,...,0.0,both,Regularly busy,Most orders,33.0,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,30.0,6.0,17.0,...,1.0,both,Regularly busy,Average orders,33.0,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,2.0,0.0,21.0,...,0.0,both,Busiest days,Average orders,6.0,New customer,7.930208,Low spender,8.0,Frequent customer
3,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,1.0,3.0,13.0,...,0.0,both,Least busy,Most orders,4.0,New customer,4.972414,Low spender,9.0,Frequent customer
4,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,3.0,4.0,17.0,...,1.0,both,Least busy,Average orders,4.0,New customer,4.972414,Low spender,9.0,Frequent customer
5,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1701441.0,777.0,16.0,1.0,7.0,...,0.0,both,Busiest days,Least orders,27.0,Regular customer,6.935398,Low spender,11.0,Regular customer
6,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1871483.0,825.0,3.0,2.0,14.0,...,0.0,both,Regularly busy,Most orders,10.0,New customer,5.957576,Low spender,19.0,Regular customer
7,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1290456.0,910.0,12.0,3.0,10.0,...,0.0,both,Least busy,Most orders,13.0,Regular customer,6.680000,Low spender,6.0,Frequent customer
8,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,369558.0,1052.0,10.0,1.0,20.0,...,0.0,both,Busiest days,Average orders,21.0,Regular customer,7.162500,Low spender,9.0,Frequent customer
9,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,589712.0,1052.0,15.0,1.0,12.0,...,1.0,both,Busiest days,Most orders,21.0,Regular customer,7.162500,Low spender,9.0,Frequent customer


In [24]:
#Check the count of each value in the column
ords_prods_merge['Order frequency'].value_counts(dropna = False)

Order frequency
Frequent customer        22869689
Regular customer          7020108
Non-frequent customer     2752653
NaN                            11
Name: count, dtype: int64

In [ ]:
# export merged pickle file.
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))